In [1]:
import re
from collections import Counter

In [2]:
# function to tokenise words
def words(document):
    "Convert text to lower case and tokenise the document"
    return re.findall(r'\w+', document.lower())

In [9]:
# create a frequency table of all the words of the document
import requests
url = "https://cdn.upgrad.com/UpGrad/temp/6ab6e37f-147a-4388-9cf2-254151347ded/big.txt"
bigTxt = requests.get(url)
bigTxt = bigTxt.text
all_words = Counter(words(bigTxt))

In [10]:
# check frequency of a random word, say, 'chair'
all_words['chair']

135

In [11]:
# look at top 10 frequent words
all_words.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [12]:
def edits_one(word):
    "Create all edits that are one edit away from `word`."
    alphabets    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])                   for i in range(len(word) + 1)]
    deletes    = [left + right[1:]                       for left, right in splits if right]
    inserts    = [left + c + right                       for left, right in splits for c in alphabets]
    replaces   = [left + c + right[1:]                   for left, right in splits if right for c in alphabets]
    transposes = [left + right[1] + right[0] + right[2:] for left, right in splits if len(right)>1]
    return set(deletes + inserts + replaces + transposes)

In [13]:
def edits_two(word):
    "Create all edits that are two edits away from `word`."
    return (e2 for e1 in edits_one(word) for e2 in edits_one(e1))

In [14]:
def known(words):
    "The subset of `words` that appear in the `all_words`."
    return set(word for word in words if word in all_words)

possible_corrections

- It first checks if the word is correct or not, i.e. if the word typed by the user is a present in the dictionary or not. If the word is present, it returns no spelling suggestions since it is already a correct dictionary word.

- If the user types a word which is not a dictionary word, then it creates a list of all the known words that are one edit distance away. If there are no valid words in the list created by edits_one() only then this function fetches a list of all known words that are two edits away from the input word

- If there are no known words that are two edits away, then the function returns the  original input word. This means that there are no alternatives that the spell corrector could find. Hence, it simply returns the original word.

In [27]:
def possible_corrections(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits_one(word)) or known(edits_two(word)) or [word])

prob 
- The function returns the probability of an input word. This is exactly why you need a seed document instead of a dictionary.

In [16]:
def prob(word, N=sum(all_words.values())): 
    "Probability of `word`: Number of appearances of 'word' / total number of tokens"
    return all_words[word] / N

In [ ]:
print(len(set(edits_one("monney"))))
print(edits_one("monney"))

442
{'monneo', 'monnhy', 'gmonney', 'mnonney', 'mronney', 'moncney', 'mnney', 'eonney', 'konney', 'mwonney', 'mohney', 'monngy', 'qmonney', 'zmonney', 'moqney', 'mononey', 'motney', 'monnuey', 'minney', 'monneyg', 'monneyn', 'lmonney', 'mdonney', 'dmonney', 'monjey', 'monnfy', 'honney', 'menney', 'moneney', 'uonney', 'movney', 'muonney', 'monmney', 'monnjy', 'mopney', 'mofney', 'molney', 'montey', 'monnew', 'moznney', 'mlnney', 'monpney', 'mmonney', 'monnmy', 'monhney', 'monniy', 'monneyd', 'monneya', 'meonney', 'monsney', 'mowney', 'monneay', 'mvnney', 'msnney', 'mvonney', 'monnvy', 'monne', 'monnley', 'monneuy', 'lonney', 'monlney', 'monfney', 'monoey', 'monndey', 'zonney', 'molnney', 'pmonney', 'monnexy', 'monnea', 'mosnney', 'monhey', 'maonney', 'tmonney', 'jmonney', 'mocney', 'monbey', 'mtonney', 'monnyy', 'monnny', 'mondney', 'monjney', 'mocnney', 'tonney', 'monnecy', 'monnef', 'moyney', 'monneyy', 'amonney', 'msonney', 'mqnney', 'monuney', 'mokney', 'monnewy', 'mmnney', 'manney'

In [18]:
print(known(edits_one("monney")))

{'monkey', 'money'}


In [ ]:
# Let's look at words that are two edits away
print(len(set(edits_two("monney"))))
print(known(edits_one("monney")))

90902
set()


In [26]:
# Let's look at possible corrections of a word
print(possible_corrections("emfasize"))

{'emphasize'}


In [21]:
# Let's look at probability of a word
print(prob("money"))
print(prob("monkey"))

0.0002922233626303688
5.378344097491451e-06


In [22]:
def spell_check(word):
    "Print the most probable spelling correction for `word` out of all the `possible_corrections`"
    correct_word = max(possible_corrections(word), key=prob)
    if correct_word != word:
        return "Did you mean " + correct_word + "?"
    else:
        return "Correct spelling."

In [23]:
# test spell check
print(spell_check("monney"))

Did you mean money?
